In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.svm import SVC
import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [2]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
print (len(dataDict))

7681


In [3]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

In [4]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [5]:
df = pd.DataFrame.from_dict(dataDict, orient='index')
df['date'] = pd.to_datetime(df['date']).dt.date
df.reset_index(inplace = True)
df.rename(columns={'index':'id'}, inplace=True)
print (df.shape)
df.head()

(7681, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a29c445588c132954d1973a,정치,2017-12-07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{중앙, 대변인, 도화선, 조선반도, 미국, 고위, 핵전쟁, 북한}"
1,5a29c445588c132954d1973b,정치,2017-12-07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{정부안, 국민의당, 호남, 의원, 증액, 예산안, 지역구}"
2,5a29c445588c132954d1973c,정치,2017-12-07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{국정원장, 구속영장 청구, 정기국회, 혐의, 의원, 조사, 검찰}"
3,5a29c445588c132954d1973d,정치,2017-12-07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{지명, 감사원장, 후보자, 공직 사회, 법관, 생활}"
4,5a29c445588c132954d1973e,정치,2017-12-07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{훈련, 공장, 삼지연, 사무차장, 김정은, 접경, 시찰, 펠트먼, 양강도}"


In [6]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [7]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    pos = ['/'.join(t) for t in pos if not t[0] in stopwords]
    return pos

In [8]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['category'])
le.classes_

array(['IT/과학', '경제', '사회', '생활/문화', '세계', '스포츠', '연예', '정치'], dtype=object)

In [10]:
w2v_docs = list()
for idx in tqdm(df.index):
    text = df.loc[idx,'title']+'.\n'+df.loc[idx,'mainText']
    pos = nav_tokenizer(text)
    label = df.loc[idx, 'category']
    label = le.transform([label])
    w2v_docs.append(TaggedDocument(pos, label))

100%|██████████| 7681/7681 [03:15<00:00, 39.25it/s]


In [11]:
train, test = train_test_split(w2v_docs, test_size = 0.15)

In [23]:
pickle.dump(train,open('./data/pre_data/train_test_Data/pre_data_doc2vec_train_for_keywords_classification_by_mecab.pickled','wb'))
pickle.dump(test,open('./data/pre_data/train_test_Data/pre_data_doc2vec_test_for_keywords_classification_by_mecab.pickled','wb'))

## doc2vec

In [15]:
d2v_model = Doc2Vec(alpha=0.025, min_alpha=0.025, iter=10)
d2v_model.build_vocab(tqdm(train))
for epoch in tqdm(range(10)):
    d2v_model.train(tqdm(train),total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
    d2v_model.alpha -= 0.002 
    d2v_model.min_alpha = d2v_model.alpha


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6528 [00:00<?, ?it/s]

  3%|▎         | 225/6528 [00:00<00:03, 1617.35it/s]

  6%|▌         | 372/6528 [00:00<00:04, 1439.55it/s]

  7%|▋         | 444/6528 [00:00<00:04, 1227.84it/s]

  8%|▊         | 522/6528 [00:00<00:05, 1124.49it/s]

  9%|▉         | 595/6528 [00:00<00:05, 1055.23it/s]

 10%|█         | 683/6528 [00:00<00:05, 1025.85it/s]

 12%|█▏        | 765/6528 [00:00<00:05, 997.21it/s] 

 13%|█▎        | 861/6528 [00:00<00:05, 987.53it/s]

 14%|█▍        | 944/6528 [00:00<00:05, 945.10it/s]

 16%|█▌        | 1056/6528 [00:01<00:05, 938.30it/s]

 18%|█▊        | 1155/6528 [00:01<00:05, 926.59it/s]

 19%|█▉        | 1237/6528 [00:01<00:05, 907.33it/s]

 21%|██        | 1350/6528 [00:01<00:05, 917.15it/s]

 22%|██▏       | 1453/6528 [00:01<00:05, 920.43it/s]

 24%|██▍       | 1556/6528 [00:01<00:05, 920.44it/s]

 26%|██▌       | 1668/6528 [00:01<00:05, 928.76it/s]

 27%|██▋       | 1778/6528 [00:01<00:05, 934.48it/s]

 

 86%|████████▌ | 5620/6528 [00:12<00:01, 467.74it/s]


100%|██████████| 10/10 [12:04<00:00, 72.43s/it]it/s]


In [16]:
d2v_model.save('./model/keywords_classification_naver_by_doc2vec_size100_window5_iter10_by_mecab.model')

In [14]:
d2v_model = Doc2Vec.load('./model/keywords_classification_naver_by_doc2vec_size100_window5_iter10_by_mecab.model')

In [ ]:
train = pickle.load(open('./data/pre_data/train_test_Data/pre_data_doc2vec_train_for_keywords_classification_by_mecab.pickled','rb'))
test = pickle.load(open('./data/pre_data/train_test_Data/pre_data_doc2vec_test_for_keywords_classification_by_mecab.pickled','rb'))

In [17]:
x_train = [x.words for x in train]
y_train = [x.tags for x in train]
x_test = [x.words for x in test]
y_test = [x.tags for x in test]

In [18]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([d2v_model.infer_vector(z).reshape(1,-1) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([d2v_model.infer_vector(z).reshape(1,-1) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

6528it [02:38, 41.14it/s]
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
1153it [00:27, 41.64it/s]


In [19]:
from keras.utils import np_utils
y_test2 = np_utils.to_categorical(y_test,8)
y_train2 = np_utils.to_categorical(y_train,8)

Using TensorFlow backend.
/Users/hyunyoun/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [21]:
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Dense, Embedding, embeddings, merge, Dropout
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(8, activation='sigmoid'))
model.compile(#optimizer='rmsprop',
    optimizer='adadelta',
              #loss='binary_crossentropy',
    #optimizer=SGD(lr=0.2), 
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train2, epochs=100, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model.evaluate(test_vecs_w2v, y_test2, verbose=2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


0.874241110147


In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model2 = Sequential()
model2.add(Dense(64, activation='relu', input_dim=100))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.125))
model2.add(Dense(8, activation='softmax'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model2.fit(train_vecs_w2v, y_train2,
          epochs=100, verbose=0,
          callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model2.evaluate(test_vecs_w2v, y_test2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


1153/1153 [==============================] - 0s 266us/step
0.886383347788


In [24]:
dataDict2 = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict2))

keywordsDict2 = pickle.load(open('./data/pre_data/keywords/keywords_daum.pickled','rb'))

for idx in dataDict2:
    dataDict2[idx]['extracted_keywords'] = keywordsDict2[idx]

df2 = pd.DataFrame.from_dict(dataDict2, orient='index')
df2['date'] = pd.to_datetime(df2['date']).dt.date
df2.reset_index(inplace = True)
df2.rename(columns={'index':'id'}, inplace=True)
print (df2.shape)
df2.head()

4722
(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{세보 효과, 공장, 부작용, 건강, 지역, 남성들, 연기}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{식후, 철분, 입냄새, 건강, 디스크, 점심 식사, 치아, 자세, 낮잠, 커피}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{성관계, 지인, 범행, 아들, 경찰, 남편, 앙심, 진술, 철원, 주변}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{유통업체, 롯데, 네이처리퍼블릭, 신영자 이사장, 혐의, 검찰, 징역, 매장}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{남자친구, 유치장, 쪽지, 경찰, 혐의, 과자, 범행, 폭행}"


In [30]:
daum_w2v = np.concatenate([d2v_model.infer_vector(z).reshape(1,-1) for z in tqdm(map(lambda x: x, df2.title.values+'\.n'+df2.mainText.values))])
daum_w2v = scale(daum_w2v)

model_pre = model.predict_classes(daum_w2v)
model2_pre = model2.predict_classes(daum_w2v)

4722it [00:23, 200.46it/s]
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [40]:
mp1 = le.inverse_transform(model_pre)

In [41]:
mp2 = le.inverse_transform(model2_pre)

In [44]:
(mp1==mp2).sum() / len(daum_w2v)

0.72236340533672172

In [60]:
embedding_matrix = np.zeros((len(d2v_model.wv.vocab), 100))
for i in range(len(d2v_model.wv.vocab)):
    embedding_vector = d2v_model.wv[d2v_model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [61]:
valid_size = 50  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')
# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])
embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)
# create the Keras model
k_model = Model(input=[valid_word, other_word], output=similarity)

def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = d2v_model.wv.index2word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(d2v_model.wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = d2v_model.wv.index2word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
  


Nearest to 12/SN: 11/SN, 7/SN, 6/SN, 9/SN, 8/SN, 4/SN, 14/SN, 5/SN,
Nearest to 이/VCP: 이/JKC, 알려졌/VV+EC+VX+EP, 알려져/VV+EC+VX+EC, 였/VCP+EP, 일리/NNG, 바뀌/VV, 나타났/VV+EP, 점찍/VV,
Nearest to 5/SN: 6/SN, 3/SN, 7/SN, 4/SN, 20/SN, 8/SN, 30/SN, 2/SN,
Nearest to 하/XSV: 하/VV, 건네/VV, 곁들이/VV, 당하/XSV, 했/XSV+EP, 이어지/VV, 힘쓰/VV, 꺼리/VV,
Nearest to 어/EC: 어/EF, 어야/EC, 었/EP, 옮겨/VV+EC, 반기/NNG, 자괴감/NNG, 매서워/VA+EC, 느껴/VV+EC,
Nearest to 재배포/NNP: 지분투자/NNP, Since/SL, 담보취득/NNP, 멕시코시티/NNP, 시구르드손/NNP, 바란/NNP, All/SL, 홍연주/NNP,
Nearest to 도/JX: 는/JX, 은/JX, 엔/JKB+JX, 이/JKS, 헤더/NNP, 특히/MAG, 이를테면/MAJ, 힘써야/VV+EC,
Nearest to 명/NNBC: 건/NNBC, 개/NNBC, 대/NNBC, 세/NNBC, 에이커/NNBC, 마리/NNBC, 회/NNBC, 석/NNBC,
Nearest to 며/EC: 고/JKQ, 면서도/VCP+EC, "./SY, 고/EC, 라면서/VCP+EC, ",/SY, 니/EC, 려면/EC,
Nearest to 대통령/NNG: 닫/NNG, 아즈/NNP, 행정부/NNG, 총장/NNG, 케어/NNP, 박정수/NNP, 슈퍼/NNG, 정권/NNG,
Nearest to 9/SN: 11/SN, 7/SN, 6/SN, 8/SN, 4/SN, 3/SN, 5/SN, 2/SN,
Nearest to 았/EP: 못했/VX+EP, 아/EC, 는다고/EC, 았었/EP, 아서/EC, 은/ETM, 는다는/ETM, 아야/EC,
Nearest to 하/VV: 하/XSV, 